In [ ]:
import obsidian
obsidian.__version__

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
from obsidian.parameters import ParamSpace, Param_Continuous
from obsidian.experiment import ExpDesigner

### Set up parameter space and initialize a design

In [ ]:
params = [
    Param_Continuous('X1', 0, 10),
    Param_Continuous('X2', 0, 10),
    ]

X_space = ParamSpace(params)
designer = ExpDesigner(X_space, seed=0)
X0 = designer.initialize(4, 'LHS')

X0

### Collect results (e.g. from a simulation)

In [ ]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import branin_currin

simulator = Simulator(X_space, branin_currin, name='Response', eps=0.05)
y0 = simulator.simulate(X0)
Z0 = pd.concat([X0, y0], axis=1)

Z0

In [ ]:
Z0.plot(x='Response 1', y='Response 2', kind='scatter', figsize=(4,3))

### Define the Target

In [ ]:
from obsidian.parameters import Target

target = [
    Target('Response 1', aim='max'),
    Target('Response 2', aim='min')
]

### Create a campaign to track optimization

In [ ]:
from obsidian.campaign import Campaign

In [ ]:
my_campaign = Campaign(X_space, target)
my_campaign.add_data(Z0)
my_campaign.data

### Fit an optimizer

In [ ]:
my_campaign.fit()

### Make new experiment suggestions

In [ ]:
from obsidian.constraints import InConstraint_Generic

In [ ]:
# # X1 + X2 >= 2
# optim_kwargs = {'m_batch':2, 'acquisition':[{'NEHVI':{'ref_point':[-350,0]}}], 'ineq_constraints':[InConstraint_Generic(X_space, indices=[0,1], coeff=[1,1], rhs=2)]}

# X1 + X2 <= 6 aka -X1 - X2 >= -6
optim_kwargs = {'m_batch':2, 'acquisition':[{'NEHVI':{'ref_point':[-350,0]}}], 'ineq_constraints':[InConstraint_Generic(X_space, indices=[0,1], coeff=[-1,-1], rhs=-6)]}

X_suggest, eval_suggest = my_campaign.optimizer.suggest(**optim_kwargs)

In [ ]:
pd.concat([X_suggest, eval_suggest], axis=1)

### Collect data at new suggestions

In [ ]:
y_iter1 = pd.DataFrame(simulator.simulate(X_suggest))
Z_iter1 = pd.concat([X_suggest, y_iter1, eval_suggest], axis=1)
my_campaign.add_data(Z_iter1)
my_campaign.data

### Repeat as desired

In [ ]:
for iter in range(5):
    my_campaign.fit()
    X_suggest, eval_suggest = my_campaign.optimizer.suggest(**optim_kwargs)
    y_iter = pd.DataFrame(simulator.simulate(X_suggest))
    Z_iter = pd.concat([X_suggest, y_iter, eval_suggest], axis=1)
    my_campaign.add_data(Z_iter)

In [ ]:
fig = px.scatter(my_campaign.data, x='Response 1', y='Response 2', color='Iteration')
fig.update_layout(height=300, width=400, template='ggplot2')

In [ ]:
my_campaign.data